In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import pickle

# Configurações
WINDOW_SIZE = 20  # Tamanho da janela deslizante (número de timesteps)
STEP_SIZE = 5     # Passo entre janelas consecutivas
FEATURES = ['R_PositionX', 'R_PositionY', 'R_PositionZ',
            'R_RotationX', 'R_RotationY', 'R_RotationZ', 'R_RotationW',
            'R_CurlIndex', 'R_CurlMiddle', 'R_CurlRing', 'R_CurlPinky', 'R_CurlThumb']
TARGET = 'Mudra'

# Função para carregar e preparar os dados
def load_and_prepare_data(file_path):
    # Carregar os dados
    df = pd.read_csv(file_path)

    # Verificar se há valores NaN nos dados
    print(df.isna().sum())

    # Preencher valores NaN ou remover linhas com valores NaN
    df = df.dropna()  # ou df = df.fillna(method='ffill')

    # Verificar valores extremos
    for feature in FEATURES:
      print(f"{feature}: Min={df[feature].min()}, Max={df[feature].max()}")

    # Converter para tipo numérico se necessário
    for feature in FEATURES:
        df[feature] = pd.to_numeric(df[feature], errors='coerce')

    # Codificar a classe alvo
    label_encoder = LabelEncoder()
    df[TARGET] = label_encoder.fit_transform(df[TARGET])
    num_classes = len(label_encoder.classes_)
    class_names = label_encoder.classes_

    return df, num_classes, class_names, label_encoder

# Função para criar janelas deslizantes
def create_sliding_windows(data, window_size, step_size):
    X = []
    y = []

    for i in range(0, len(data) - window_size, step_size):
        window = data.iloc[i:i + window_size]
        X.append(window[FEATURES].values)
        # Usamos o label do último timestep como alvo
        y.append(window[TARGET].iloc[-1])

    return np.array(X), np.array(y)

# Função para construir o modelo
def build_model(input_shape, num_classes):
    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Função principal para treinar o modelo
def train_vr_gesture_predictor(file_path):
    # Carregar e preparar os dados
    df, num_classes, class_names, label_encoder = load_and_prepare_data(file_path)

    # Normalizar os dados de características
    scaler = StandardScaler()
    df[FEATURES] = scaler.fit_transform(df[FEATURES])

    # Criar janelas deslizantes
    X, y = create_sliding_windows(df, WINDOW_SIZE, STEP_SIZE)

    # Dividir em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Converter os rótulos para one-hot encoding
    y_train_cat = to_categorical(y_train, num_classes=num_classes)
    y_test_cat = to_categorical(y_test, num_classes=num_classes)

    # Construir o modelo
    model = build_model((WINDOW_SIZE, len(FEATURES)), num_classes)

    # Early stopping para evitar overfitting
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    # Treinar o modelo
    history = model.fit(
        X_train, y_train_cat,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )

    # No final da função de treinamento, adicione:
    model.save("vr_gesture_model.h5")
    with open("vr_gesture_scaler.pkl", 'wb') as f:
      pickle.dump(scaler, f)
    with open("vr_gesture_encoder.pkl", 'wb') as f:
      pickle.dump(label_encoder, f)

    # Avaliar o modelo
    loss, accuracy = model.evaluate(X_test, y_test_cat, verbose=0)
    print(f"Acurácia do teste: {accuracy:.4f}")

    # Plotar histórico de treinamento
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Treino')
    plt.plot(history.history['val_accuracy'], label='Validação')
    plt.title('Acurácia do Modelo')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Treino')
    plt.plot(history.history['val_loss'], label='Validação')
    plt.title('Perda do Modelo')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return model, scaler, label_encoder

# Função para fazer previsões em tempo real (simulação)
def predict_gesture_realtime(model, scaler, label_encoder, new_data):
    """
    Função para prever gestos em tempo real.

    Args:
        model: Modelo treinado
        scaler: Scaler ajustado
        label_encoder: Codificador de rótulos
        new_data: DataFrame com as últimas WINDOW_SIZE leituras

    Returns:
        Gesto previsto e probabilidades
    """
    # Preparar os dados
    features_data = new_data[FEATURES].values
    normalized_data = scaler.transform(features_data)

    # Reshape para o formato esperado pelo modelo
    input_data = normalized_data.reshape(1, WINDOW_SIZE, len(FEATURES))

    # Fazer a previsão
    prediction_probs = model.predict(input_data)[0]
    predicted_class = np.argmax(prediction_probs)
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]

    return predicted_label, prediction_probs

# Exemplo de uso
if __name__ == "__main__":
    # Substitua pelo caminho do seu arquivo
    file_path = "combined_one_hand_data_with_classification.csv"

    # Treinar o modelo
    model, scaler, label_encoder = train_vr_gesture_predictor(file_path)

    # Salvar o modelo e os transformadores para uso posterior
    model.save("vr_gesture_model.h5")

    # Exemplo de como usar o modelo em tempo real
    # (Na implementação real, você usaria dados ao vivo do controlador VR)
    df = pd.read_csv(file_path)
    sample_window = df.iloc[-WINDOW_SIZE:].copy()

    predicted_gesture, probabilities = predict_gesture_realtime(
        model, scaler, label_encoder, sample_window
    )

    print(f"Gesto Previsto: {predicted_gesture}")

In [ ]:
import random
WINDOW_SIZE = random.choice(range(0, 57338, 20))  # from 0 to 200, step 20

sample_window = df.iloc[-WINDOW_SIZE:].copy()

predicted_gesture, probabilities = predict_gesture_realtime(
    model, scaler, label_encoder, sample_window
)

print(f"Gesto Previsto: {predicted_gesture}")